# Portfolio Optimization and Black-Litterman

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.optimize

In [2]:
def port_mean(weights, returns):
    return sum(returns * weights)

def port_var(weights, covariance):
    return np.dot(np.dot(weights, covariance), weights)

def port_mean_var(weights, returns, covariance):
    return port_mean(weights, returns), port_var(weights, covariance)

In [3]:
def solve_frontier(returns, covariance, risk_free_rate):
    def fitness(weights, returns, covariance, risk_free_rate):
        mean, var = port_mean_var(weights, returns, covariance)
        penalty = 100 * abs(mean - r)  
        return var + penalty

    frontier_mean, frontier_var = [], []
    number_of_assets = len(returns)  
    for r in np.linspace(min(returns), max(returns), num=20):  
        weights = np.ones([number_of_assets]) / number_of_assets  
        b_ = [(0, 1) for i in range(number_of_assets)]
        c_ = ({'type': 'eq', 'fun': lambda w: sum(w) - 1.0})
        optimized = scipy.optimize.minimize(fitness, weights, (returns, covariance, returns), 
                                            method='SLSQP', constraints=c_, bounds=b_)
        if not optimized.success:
            raise BaseException(optimized.message)
        frontier_mean.append(r)
        frontier_var.append(port_var(optimized.x, covariance))
    return np.array(frontier_mean), np.array(frontier_var)

In [4]:
def solve_weights(returns, covariance, risk_free_rate):
    def fitness(weights, returns, covariance, risk_free_rate):
        mean, var = port_mean_var(weights, returns, covariance)  
        util = (mean - risk_free_rate) / np.sqrt(var)  
        return 1.0 / util  
    number_of_assets = len(returns)
    weights = np.ones([number_of_assets]) / number_of_assets  
    b_ = [(0.0, 1.0) for i in range(number_of_assets)]  
    c_ = ({'type': 'eq', 'fun': lambda w: sum(w) - 1.0})  
    optimized = scipy.optimize.minimize(fitness, weights, (returns, covariance, risk_free_rate), 
                                        method='SLSQP', constraints=c_, bounds=b_)
    if not optimized.success: raise BaseException(optimized.message)
    return optimized.x

In [5]:
class Result:
    def __init__(self, weights, tan_mean, tan_var, front_mean, front_var):
        self.W=weights
        self.tan_mean=tan_mean
        self.tan_var=tan_var
        self.front_mean=front_mean
        self.front_var=front_var
        
def optimize_frontier(returns, covariance, risk_free_rate):
    weights = solve_weights(returns, covariance, risk_free_rate)
    tan_mean, tan_var = port_mean_var(weights, risk_free_rate, covariance)  
    front_mean, front_var = solve_frontier(returns, covariance, risk_free_rate)  
    return Result(weights, tan_mean, tan_var, front_mean, front_var)

def display_assets(names, R, C, color='black'):
    fig, ax = plt.subplots()
    fig.set_size_inches(10, 8)
    
    ax.scatter([C[i, i] ** .5 for i in range(len(R))], R, marker='x', color=color), ax.grid(True)  # draw assets
    for i in range(len(R)): 
        ax.text(C[i, i] ** .5, R[i], '  %s' % names[i], verticalalignment='center', color=color) # draw labels

def display_frontier(result, label=None, color='black'):
    fig, ax = plt.subplots()
    fig.set_size_inches(10, 8)

    ax.text(result.tan_var ** .5, result.tan_mean, '   tangent', verticalalignment='center', color=color)
    ax.scatter(result.tan_var ** .5, result.tan_mean, marker='o', color=color), ax.grid(True)
    ax.plot(result.front_var ** .5, result.front_mean, label=label, color=color), ax.grid(True)
    
    plt.show()

## Load historical prices

In [6]:
def load_data():
    symbols = ['XOM', 'AAPL', 'MSFT', 'JNJ', 'GE', 'GOOG', 'CVX', 'PG', 'WFC']
    cap = {'XOM': 403.02e9, 'AAPL': 392.90e9, 'MSFT': 283.60e9, 'JNJ': 243.17e9, 'GE': 236.79e9,
           'GOOG': 292.72e9, 'CVX': 231.03e9, 'PG': 214.99e9, 'WFC': 218.79e9}
    prices_out, caps_out = [], []
    for s in symbols:
        dataframe = pd.read_csv('data/%s.csv' % s, index_col=None, parse_dates=['date'])
        prices = list(dataframe['close'])[-500:] # trailing window 500 days
        prices_out.append(prices)
        caps_out.append(cap[s])
    return symbols, prices_out, caps_out

names, prices, caps = load_data()
number_of_assets = len(names)

## Estimate assets historical return and covariances

In [7]:
def assets_historical_returns_and_covariances(prices):
    prices = np.matrix(prices)
    
    # create matrix of historical returns
    rows, cols = prices.shape
    returns = np.empty([rows, cols - 1])
    for r in range(rows):
        for c in range(cols - 1):
            p0, p1 = prices[r, c], prices[r, c + 1]
            returns[r, c] = (p1 / p0) - 1

    # calculate returns
    expreturns = np.array([])
    for r in range(rows):
        expreturns = np.append(expreturns, np.mean(returns[r]))
    
    # calculate covariances
    covars = np.cov(returns)

    expreturns = (1 + expreturns) ** 252 - 1
    covars = covars * 252
    return expreturns, covars


risk_free_rate = .015  
weights = np.array(caps) / sum(caps) 
returns, covariance = assets_historical_returns_and_covariances(prices)

## Asset Returns and Weights

In [8]:
pd.DataFrame({'Return': returns, 'Weight (based on market cap)': weights}, index=names).T

,XOM,AAPL,MSFT,JNJ,GE,GOOG,CVX,PG,WFC
Return,0.073598,0.130637,0.173537,0.144994,0.141584,0.335016,0.092902,0.113168,0.271355
Weight (based on market cap),0.160119,0.156098,0.112673,0.096611,0.094076,0.116297,0.091787,0.085415,0.086925


## Asset Covariances

In [9]:
pd.DataFrame(covariance, columns=names, index=names)

,XOM,AAPL,MSFT,JNJ,GE,GOOG,CVX,PG,WFC
XOM,0.039328,0.022475,0.028206,0.019752,0.036940,0.026942,0.038551,0.018362,0.043315
AAPL,0.022475,0.092579,0.024796,0.011670,0.028036,0.033515,0.028466,0.011625,0.037050
MSFT,0.028206,0.024796,0.051422,0.016977,0.032910,0.026441,0.031061,0.013834,0.040643
JNJ,0.019752,0.011670,0.016977,0.019612,0.021217,0.015671,0.021173,0.013302,0.027076
GE,0.036940,0.028036,0.032910,0.021217,0.059707,0.031332,0.041003,0.019532,0.054359
GOOG,0.026942,0.033515,0.026441,0.015671,0.031332,0.067434,0.030500,0.015244,0.039331
CVX,0.038551,0.028466,0.031061,0.021173,0.041003,0.030500,0.051264,0.019744,0.048712
PG,0.018362,0.011625,0.013834,0.013302,0.019532,0.015244,0.019744,0.025354,0.022659
WFC,0.043315,0.037050,0.040643,0.027076,0.054359,0.039331,0.048712,0.022659,0.090545


## Mean-Variance Optimization (based on historical returns)

In [10]:
res1 = optimize_frontier(returns, covariance, risk_free_rate)

display_assets(names, returns, covariance, color='blue')
display_frontier(res1, color='blue')
# todo  pd.xlabel('variance $\sigma$'), pd.ylabel('mean $\mu$'), pd.show()
pd.DataFrame({'Weight': res1.W}, index=names).T

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,XOM,AAPL,MSFT,JNJ,GE,GOOG,CVX,PG,WFC
Weight,5.478835e-17,0.0,0.0,0.450865,0.0,0.528449,1.714509e-16,6.824036e-18,0.020686


## Black-Litterman Reverse Optimization

In [11]:
# Calculate portfolio historical return and variance
mean, var = port_mean_var(weights, returns, covariance)

risk_aversion_coefficient = (mean - risk_free_rate) / var  
port_equil_excess_returns = np.dot(np.dot(risk_aversion_coefficient, covariance), weights)  

## Mean-variance Optimization (based on equilibrium returns)

In [12]:
res2 = optimize_frontier(port_equil_excess_returns+risk_free_rate, covariance, risk_free_rate)

display_assets(names, returns, covariance, color='red')
display_frontier(res1, label='Historical returns', color='red')
display_assets(names, port_equil_excess_returns+risk_free_rate, covariance, color='green')
display_frontier(res2, label='Implied returns', color='green')
# todo pd.xlabel('variance $\sigma$'), pd.ylabel('mean $\mu$'), pd.legend(), pd.show()
pd.DataFrame({'Weight': res2.W}, index=names).T

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,XOM,AAPL,MSFT,JNJ,GE,GOOG,CVX,PG,WFC
Weight,0.160431,0.15617,0.112448,0.097429,0.093801,0.116217,0.091825,0.084676,0.087003


## Determine views to the equilibrium returns and prepare views (Q) and link (P) matrices

In [13]:
def create_views_and_link_matrix(names, views):
    r, c = len(views), len(names)
    Q = [views[i][3] for i in range(r)]  # view matrix
    P = np.zeros([r, c])
    nameToIndex = dict()
    for i, n in enumerate(names):
        nameToIndex[n] = i
    for i, v in enumerate(views):
        name1, name2 = views[i][0], views[i][2]
        P[i, nameToIndex[name1]] = +1 if views[i][1] == '>' else -1
        P[i, nameToIndex[name2]] = -1 if views[i][1] == '>' else +1
    return np.array(Q), P

views = [('MSFT', '>', 'GE', 0.02),
         ('AAPL', '<', 'JNJ', 0.02)]

Q, P = create_views_and_link_matrix(names, views)
print('Views Matrix')
pd.DataFrame({'Views':Q})
print('Link Matrix')
pd.DataFrame(P)

Views Matrix
Link Matrix


,0,1,2,3,4,5,6,7,8
0,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0
1,0.0,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


## Optimization based on Equilibrium returns with adjusted views

In [14]:
tau = .025 

# Calculate omega - uncertainty matrix about views
omega = np.dot(np.dot(np.dot(tau, P), covariance), np.transpose(P))  
# Calculate equilibrium excess returns with views incorporated
sub_a = np.linalg.inv(np.dot(tau, covariance))
sub_b = np.dot(np.dot(np.transpose(P), np.linalg.inv(omega)), P)
sub_c = np.dot(np.linalg.inv(np.dot(tau, covariance)), port_equil_excess_returns)
sub_d = np.dot(np.dot(np.transpose(P), np.linalg.inv(omega)), Q)
Pi_adj = np.dot(np.linalg.inv(sub_a + sub_b), (sub_c + sub_d))

res3 = optimize_frontier(port_equil_excess_returns+risk_free_rate, covariance, risk_free_rate)

display_assets(names, port_equil_excess_returns+risk_free_rate, covariance, color='green')
display_frontier(res2, label='Implied returns', color='green')
display_assets(names, Pi_adj+risk_free_rate, covariance, color='blue')
display_frontier(res3, label='Implied returns (adjusted views)', color='blue')
# todo pd.xlabel('variance $\sigma$'), pd.ylabel('mean $\mu$'), pd.legend(), pd.show()
pd.DataFrame({'Weight': res2.W}, index=names).T



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,XOM,AAPL,MSFT,JNJ,GE,GOOG,CVX,PG,WFC
Weight,0.160431,0.15617,0.112448,0.097429,0.093801,0.116217,0.091825,0.084676,0.087003
